In [59]:
import pyodbc
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
spark = SparkSession.builder.getOrCreate()
connection_string = (
    r'Driver={ODBC Driver 18 for SQL Server};'
    r'Server=localhost;'
    r'Database=DATA_WAREHOUSE;'
    r'Trusted_Connection=yes;'
    r'TrustServerCertificate=yes;'
)

my_odbc_connection = pyodbc.connect(connection_string)

my_odbc_cursor = my_odbc_connection.cursor()

# my_odbc_cursor.execute("SELECT * FROM [DATA_WAREHOUSE].[STANDALONE_DATASETS].[MARKETING_WAR];")

my_insert_query = f"""INSERT INTO [STANDALONE_DATASETS].[MARKETING_WAR]
           ([visitors]
           ,[revenue]
           ,[marketing_spend]
           ,[promotion]
           ,[present_date]
           ,[present_day_name]
           ,[present_week_number]
           ,[present_week_id]
           ,[present_month_number]
           ,[present_month_id]
           ,[present_year]
           ,[created_by]
           ,[created_date]
           ,[updated_by]
           ,[updated_date])VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);"""



In [60]:
my_marketing_df = spark.read.option('header', True).csv("./marketing_war_data.csv").na.drop(subset=["Week"])

In [61]:
my_update_query_list = []

for current_entry in my_marketing_df.collect():
    Date = datetime.strptime(current_entry["Date"], "%d-%m-%Y")
    my_update_query_list.append(
        (
            int(current_entry["Visitors"]),
            int(current_entry["Revenue"]),
            float(current_entry["Marketing Spend"]),
            str(current_entry["Promo"]).lower().title(),
            Date,
            str(current_entry["Day_Name"]).lower().title(),
            int(current_entry["Week"]),
            int(current_entry["Week_ID"]),
            int(current_entry["Month"]),
            int(current_entry["Month_ID"]),
            int(current_entry["Year"]),
            "MANUAL_LOAD",
            datetime.now(),
            "MANUAL_LOAD",
            datetime.now(),
        )
    )

my_odbc_cursor.executemany(my_insert_query, my_update_query_list)
my_odbc_cursor.commit()